In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#packages to deal with fuzzy words
import fuzzywuzzy
from fuzzywuzzy import process
import charset_normalizer

In [ ]:
#this file list all name of citys for flight poins
airports= pd.read_csv('./airports.csv')
airports

In [ ]:
data= pd.read_csv('./airlines_reviews_sen_cleaned.csv')

In [ ]:
airports['City Name'] = airports['City Name'].str.lower()
airports['Airport Name'] = airports['Airport Name'].str.lower()

In [ ]:
cols=['Start_point','End_point']
for col in cols:
  # convert to lower case
  data[col] = data[col].str.lower()
  # remove trailing white spaces
  data[col] = data[col].str.strip()

In [ ]:
series = pd.Series(list(airports["City Name"]), index=airports["City Name"])
series_t = pd.Series(list(airports["City Name"]), index=airports["Airport Name"])
dic1 = series.to_dict()
dic2=series_t.to_dict()
merged_dict = dic1.copy()
merged_dict.update(dic2)
dic=merged_dict

In [ ]:
data["End_point"]= data["End_point"].apply(lambda x: dic[x] if x in dic else x)

In [ ]:
data["Start_point"]= data["Start_point"].apply(lambda x: dic[x] if x in dic else x)

In [ ]:
matches = fuzzywuzzy.process.extract("zurich ", en_list, limit=10, scorer= fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('zurich', 100),
 ('zurich ', 100),
 ('munich', 67),
 ('zrh', 67),
 ('norwich', 62),
 ('münich', 55),
 ('turin', 55),
 ('durham', 50),
 ('minich', 50),
 ('murcia', 50)]

In [ ]:
replace_matches_in_column(df=data, column='Start_point', string_to_match="zurich ", min_ratio= 75)

All done!


In [ ]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio):
    # get a list of unique strings
    strings = df[column].unique()

    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings,
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches
    df.loc[rows_with_matches, column] = string_to_match

    # let us know the function's done
    print("All done!")

In [ ]:
data['Airlines']= data['Airlines'].map({'finnair':'Finnair', 'sas-scandinavian-airlines':'SAS',
       'swiss-international-air-lines':'SWISS', 'klm-royal-dutch-airlines':'KLM',
       'turkish-airlines': 'Turkish-Airlines','lufthansa':'Lufthansa', 'air-france': 'Air-France', 'british-airways': 'British-Airways',
       'wizz-air': 'Wizz-Air', 'easyjet': 'Easyjet', 'Ryanair': 'Ryanair'})

In [ ]:
data

,Type_Of_Traveller,Seat_Type,Route,Date_Flown,Seat_Comfort,Staff_Service,F&B,Inflight_Entertainment,Ground_Service,Value_For_Money,Recommended,Airlines,Cleaned Reviews,Lemma,Sentiment,Analysis,Start_point,End_point
0,Couple Leisure,Economy Class,Paris to Hong Kong via Helsinki,May 2023,2.0,3.0,1.0,1.0,1.0,2.0,No,Finnair,Check in in Riga was fast friendly Although n...,Check Riga fast friendly Although noisy ATR ...,0.9442,Positive,paris,hong kong
1,Business,Economy Class,Heathrow to Helsinki,May 2023,3.0,3.0,1.0,1.0,1.0,1.0,No,Finnair,Unhelpful untrained check in staff that does ...,Unhelpful untrained check staff know anythin...,0.5994,Positive,heathrow,helsinki
2,Family Leisure,Economy Class,Helsinki to Miami,January 2023,3.0,3.0,2.0,2.0,3.0,4.0,No,Finnair,Very disappointing customer experience with F...,disappointing customer experience Finnair Fl...,-0.4215,Negative,helsinki,miami
3,Solo Leisure,Business Class,Doha to Stockholm,April 2023,3.0,3.0,3.0,5.0,4.0,3.0,Yes,Finnair,Was very disappointed with my recent experien...,disappointed recent experience customer serv...,-0.8689,Negative,doha,stockholm
4,Family Leisure,Economy Class,Helsinki to Bangkok,December 2022,3.0,1.0,1.0,1.0,1.0,2.0,No,Finnair,No champagne when boarding Apparently some ru...,champagne board Apparently rule Finnair serv...,0.6808,Positive,helsinki,bangkok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9570,Family Leisure,Economy Class,Malaga to Newcastle,January 2018,2.0,3.0,1.0,1.0,1.0,3.0,No,Ryanair,I am disgusted at the amount I had to pay bec...,disgust amount pay forget check online round...,-0.7645,Negative,malaga,newcastle
9571,Couple Leisure,Economy Class,East Midlands to Berlin,December 2017,3.0,4.0,3.0,1.0,2.0,3.0,No,Ryanair,I have over flights with this company and sti...,flight company still current booking mean qu...,-0.7584,Negative,east midlands,berlin
9572,Business,Economy Class,Ponta Delgada to Porto,December 2017,3.0,4.0,2.0,1.0,4.0,4.0,Yes,Ryanair,Malaga to Newcastle We had a few issues at th...,Malaga Newcastle issue check family travel r...,-0.5984,Negative,ponta delgada,porto
9573,Family Leisure,Economy Class,Milan to Catania,December 2017,3.0,3.0,1.0,1.0,2.0,3.0,Yes,Ryanair,I went on a short city break with my boyfrien...,go short city break boyfriend New Year fly E...,0.0000,Neutral,milan,catania
